# API 데이터 불러오기 테스트

In [46]:
import requests, json

URL = "http://apis.data.go.kr/6260000/FoodService/getFoodKr"
param = {
    "ServiceKey" : "dCxHweONFM65igJwj+AbNCd3hs+qqixeRE/9VGbJvRdlYb+b/NMK332mLzdVWNe8IaFKis4L184Qr9rDjFGnNA==",
    "pageNo" : 1,
    "numOfRows" : 10,
    "resultType" : "json"
}

req = requests.get(URL, params = param)
data = req.json()
print(    json.dumps(data, indent=2, ensure_ascii=False)       )

{
  "getFoodKr": {
    "header": {
      "code": "00",
      "message": "NORMAL_CODE"
    },
    "item": [
      {
        "UC_SEQ": 70,
        "MAIN_TITLE": "만드리곤드레밥",
        "GUGUN_NM": "강서구",
        "LAT": 35.177387,
        "LNG": 128.95245,
        "PLACE": "만드리곤드레밥",
        "TITLE": "만드리곤드레밥",
        "SUBTITLE": "",
        "ADDR1": "강서구 공항앞길 85번길 13",
        "ADDR2": "",
        "CNTCT_TEL": "051-941-3669",
        "HOMEPAGE_URL": "",
        "USAGE_DAY_WEEK_AND_TIME": "10:00-20:00\n(19:50 라스트오더)",
        "RPRSNTV_MENU": "돌솥곤드레정식, 단호박오리훈제",
        "MAIN_IMG_NORMAL": "https://www.visitbusan.net/uploadImgs/files/cntnts/20191209162810545_ttiel",
        "MAIN_IMG_THUMB": "https://www.visitbusan.net/uploadImgs/files/cntnts/20191209162810545_thumbL",
        "ITEMCNTNTS": "곤드레밥에는 일반적으로 건조 곤드레나물이 사용되는데,\n이곳은 생 곤드레나물을 사용하여 돌솥밥을 만든다.\n된장찌개와 함께 열 가지가 넘는 반찬이 제공되는 돌솥곤드레정식이 인기있다\n"
      },
      {
        "UC_SEQ": 72,
        "MAIN_TITLE": "만호갈미샤브샤브",
        "GUGUN_NM": "강서구",
  

# 기본 데이터 지도 시각화

In [1]:
import requests, json
import folium

URL = "http://apis.data.go.kr/6260000/FoodService/getFoodKr"
param = {
    "ServiceKey" : "dCxHweONFM65igJwj+AbNCd3hs+qqixeRE/9VGbJvRdlYb+b/NMK332mLzdVWNe8IaFKis4L184Qr9rDjFGnNA==",
    "pageNo" : 1,
    "numOfRows" : 100,
    "resultType" : "json"
}

req = requests.get(URL, params = param)
data = req.json()

items = data["getFoodKr"]["item"]

m = folium.Map(location=[35.1796, 129.0756], zoom_start = 11)


icon_dict = {
    "한식" : ("utensils", "green"),
    "중식" : ("bowl-rice", "red"),
    "양식" : ("pizza-slice", "blue"),
    "고기/구이" : ("drumstick-bite", "orange"),
    "해산물" : ("fish", "cadetblue"),
    "기타" : ("store", "purple")
}


for it in items:

    lat = float(it.get("LAT"))
    lng = float(it.get("LNG"))
    
    title = it.get("TITLE")
    addr = it.get("ADDR1", "")
    tel = it.get("CNTCT_TEL", "")
    menus = it.get("RPRSNTV_MENU", "")
    thumb = it.get("MAIN_IMG_THUMB", "")

    if any(x in menus for x in ["비빔밀면", "밀면", "돌솥"]):
        icon_name, color = icon_dict["한식"]        
    elif any(x in menus for x in ["중국", "짜장", "짬뽕"]):
        icon_name, color = icon_dict["중식"]        
    elif any(x in menus for x in ["파스타", "스테이크", "피자"]):
        icon_name, color = icon_dict["양식"]        
    elif any(x in menus for x in ["회", "조개", "낙지", "붕어"]):
        icon_name, color = icon_dict["해산물"]        
    else:
        icon_name, color = icon_dict["기타"]
    
    folium.Marker(
        location=[ lat ,  lng],
        tooltip = title,
        popup = f"<h4>{title}</h4>연락처 : {tel}<br>주소 : {addr}<br>대표메뉴 : {menus}<br><img src='{thumb}' width='200'>",
        icon=folium.Icon(color=color, icon=icon_name, prefix="fa")
    ).add_to(m)

m.save("busan_foods.html")
m


# 지도 업데이트 Ver 1.2

In [6]:
import requests, json
import folium

URL = "http://apis.data.go.kr/6260000/FoodService/getFoodKr"
param = {
    "ServiceKey" : "dCxHweONFM65igJwj+AbNCd3hs+qqixeRE/9VGbJvRdlYb+b/NMK332mLzdVWNe8IaFKis4L184Qr9rDjFGnNA==",
    "pageNo" : 1,
    "numOfRows" : 10,
    "resultType" : "json"
}

req = requests.get(URL, params = param)
data = req.json()

items = data["getFoodKr"]["item"]

m = folium.Map(location=[35.1796, 129.0756], zoom_start = 11)


icon_dict = {
    "한식" : ("utensils", "green"),
    "중식" : ("bowl-rice", "red"),
    "양식" : ("pizza-slice", "blue"),
    "고기/구이" : ("drumstick-bite", "orange"),
    "해산물" : ("fish", "cadetblue"),
    "기타" : ("store", "purple")
}

#업종별 토글 버튼으로 그룹 생성하기(LayerGroup)
layers = {}
for k in icon_dict.keys():
    layers[k] = folium.FeatureGroup(name=k).add_to(m)

for it in items:

    lat = float(it.get("LAT"))
    lng = float(it.get("LNG"))
    
    title = it.get("TITLE")
    addr = it.get("ADDR1", "")
    tel = it.get("CNTCT_TEL", "")
    menus = it.get("RPRSNTV_MENU", "")
    thumb = it.get("MAIN_IMG_THUMB", "")

    if any(x in menus for x in ["비빔밀면", "밀면", "돌솥"]):
        icon_name, color = icon_dict["한식"]        
    elif any(x in menus for x in ["중국", "짜장", "짬뽕"]):
        icon_name, color = icon_dict["중식"]        
    elif any(x in menus for x in ["파스타", "스테이크", "피자"]):
        icon_name, color = icon_dict["양식"]        
    elif any(x in menus for x in ["회", "조개", "낙지", "붕어"]):
        icon_name, color = icon_dict["해산물"]        
    else:
        icon_name, color = icon_dict["기타"]
    
    folium.Marker(
        location=[ lat ,  lng],
        tooltip = title,
        popup = f"<h4>{title}</h4>연락처 : {tel}<br>주소 : {addr}<br>대표메뉴 : {menus}<br><img src='{thumb}' width='200'>",
        icon=folium.Icon(color=color, icon=icon_name, prefix="fa")
    ).add_to(m)

folium.LayerControl().add_to(m)    

m.save("busan_foods.html")
m


# 지도 업데이트 Ver 2.0

In [14]:
import requests, json
import folium
from folium.plugins import MarkerCluster, BeautifyIcon

URL = "http://apis.data.go.kr/6260000/FoodService/getFoodKr"
param = {
    "ServiceKey" : "dCxHweONFM65igJwj+AbNCd3hs+qqixeRE/9VGbJvRdlYb+b/NMK332mLzdVWNe8IaFKis4L184Qr9rDjFGnNA==",
    "pageNo" : 1,
    "numOfRows" : 50,
    "resultType" : "json"
}

req = requests.get(URL, params = param)
data = req.json()

items = data["getFoodKr"]["item"]

m = folium.Map(location=[35.1796, 129.0756], zoom_start = 11)


icon_dict = {
    "한식" : ("utensils", "green"),
    "중식" : ("bowl-rice", "red"),
    "양식" : ("pizza-slice", "blue"),
    "고기/구이" : ("drumstick-bite", "orange"),
    "해산물" : ("fish", "cadetblue"),
    "기타" : ("store", "purple")
}

#업종별 토글 버튼으로 그룹 생성하기(LayerGroup)
layers = {}
for k in icon_dict.keys():
    layers[k] = folium.FeatureGroup(name=k).add_to(m)

m_cluster = MarkerCluster().add_to(m)


for it in items:

    lat = float(it.get("LAT"))
    lng = float(it.get("LNG"))
    
    title = it.get("TITLE")
    addr = it.get("ADDR1", "")
    tel = it.get("CNTCT_TEL", "")
    menus = it.get("RPRSNTV_MENU", "")
    thumb = it.get("MAIN_IMG_THUMB", "")

    if any(x in menus for x in ["비빔밀면", "밀면", "돌솥"]):
        icon_name, color = icon_dict["한식"]        
    elif any(x in menus for x in ["중국", "짜장", "짬뽕"]):
        icon_name, color = icon_dict["중식"]        
    elif any(x in menus for x in ["파스타", "스테이크", "피자"]):
        icon_name, color = icon_dict["양식"]        
    elif any(x in menus for x in ["회", "조개", "낙지", "붕어"]):
        icon_name, color = icon_dict["해산물"]        
    else:
        icon_name, color = icon_dict["기타"]

    if lat and lng:
        folium.Marker(
            location=[ lat ,  lng],
            tooltip = title,
            popup = f"<h4>{title}</h4>연락처 : {tel}<br>주소 : {addr}<br>대표메뉴 : {menus}<br><img src='{thumb}' width='200'>",
            icon=BeautifyIcon(
                icon=icon_name, 
                border_color=color,
                background_color=color,
                icon_shape='circle',
                text_color='white'
            )
        ).add_to(m_cluster)

        folium.CircleMarker(
            location=[ lat ,  lng],
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.5,
            popup = f"<h4>{title}</h4>연락처 : {tel}<br>주소 : {addr}<br>대표메뉴 : {menus}<br><img src='{thumb}' width='200'>",
        ).add_to(layers[k])

folium.LayerControl().add_to(m)    

m.save("busan_foods.html")
m
